In [45]:
!python -m pip install requests
!python -m pip install imdbpy
!python -m pip install pandas
!python -m pip install psycopg2
!python -m pip install dask

In [46]:
import numpy as np
import pandas as pd
import json
from sqlalchemy import create_engine
import psycopg2
import ast

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    host="34.30.45.126",
    port=5432,
    database="finalproject",
    user="postgres",
    password="teamrocket"
)

# Create a cursor
cur = conn.cursor()

# Create a SQLAlchemy engine
engine = create_engine('postgresql+psycopg2://', creator=lambda: conn)

In [47]:
# Define the table names for the data
movie_table_name = "movie_data"
twitter_data=['twitter_movie_score','twitter_cast_score','twitter_director_score']
reddit_data=['reddit_movie_title','reddit_director','reddit_cast_score','reddit_cast_avg_score']

# Read these data from the table as dataframes
with engine.connect() as con:
    df_movie = pd.read_sql_table(movie_table_name, con=con)
    df_twitter_movie_score = pd.read_sql_table(twitter_data[0], con=con)
    df_twitter_cast_score  = pd.read_sql_table(twitter_data[1], con=con)
    df_twitter_director_score  = pd.read_sql_table(twitter_data[2], con=con)
    df_reddit_movie_title = pd.read_sql_table(reddit_data[0], con=con)
    df_reddit_director    = pd.read_sql_table(reddit_data[1], con=con)
    df_reddit_average_score  = pd.read_sql_table(reddit_data[3], con=con)
    


In [59]:
df_twitter_director_score

,id,director_name,score,tweet_end_id,tweet_count
0,412795,Paco Bardales,0.000000,1654444654786408455,0
1,34615,Darrell James Roodt,0.000000,1654462667535396866,0
2,5757,"[""Dirk Shafer""]",0.000000,1654470303584509958,0
3,31357,Forest Whitaker,-0.552932,1654434754354135042,1313
4,19155,Milcho Manchevski,0.000000,1654558920327741442,0
...,...,...,...,...,...
15596,25624,John Singleton,0.000000,1654946491222614017,3100
15597,9360,Luis Llosa,0.000000,1653854864479330309,33
15598,377,Wes Craven,0.000000,1654930962877227012,1104
15599,653,F. W. Murnau,0.000000,1654883695453429761,168


In [56]:
df_final=df_movie
df_reddit_movie_title.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17174 entries, 0 to 17173
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   index            17174 non-null  int64  
 1   id               17174 non-null  int64  
 2   original_title   17174 non-null  object 
 3   title_l/d_ratio  17174 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 536.8+ KB


#### Standardize  the data

In [50]:
df_final['Top Cast'] = df_final['Top Cast'].apply(lambda x: [i.strip('"') for i in x.strip('][').split(',')] if isinstance(x, str) else x)
df_reddit_movie_title['id'] = df_reddit_movie_title['id'].astype('int64')
df_reddit_average_score['id'] = df_reddit_average_score['id'].astype('int64')
df_reddit_director['id'] = df_reddit_director['id'].astype('int64')
df_final['id'] = df_final['id'].astype('int64')

In [51]:
def add_twitter_cast_average_score(df_final, df_twitter_cast_score):
    # Create a function to calculate the average score for a list of cast names
    def get_average_score(cast_names):
        scores = df_twitter_cast_score.loc[df_twitter_cast_score['cast_name'].isin(cast_names), 'score']
        if len(scores) > 0:
            return scores.mean()
        else:
            return None
    # Add a new column to df_final with the average scores for each row's Top Cast
    df_final['twitter_cast_average_score'] = df_final['Top Cast'].apply(get_average_score)
    return df_final

#### Merge the data

In [52]:
if 'twitter_movie_score' not in df_final.columns:
    df_final = df_final.merge(df_twitter_movie_score[['id', 'score']], on='id', how='left', suffixes=('_movie', '_twitter'))
    df_final.rename(columns={'score': 'twitter_movie_score'}, inplace=True)

df_final=add_twitter_cast_average_score(df_final,df_twitter_cast_score)

if 'twitter_director_score' not in df_final.columns:
    df_final = df_final.merge(df_twitter_director_score[['id', 'score']], on='id', how='left', suffixes=('_director', '_twitter'))
    df_final.rename(columns={'score': 'twitter_director_score'}, inplace=True)
    
df_final

,adult,belongs_to_collection,homepage,id,imdb_id,production_countries,spoken_languages,video,keywords,backdrop_path,...,vote_average,vote_count,cast,director,New_status,rating,Top Cast,twitter_movie_score,twitter_cast_average_score,twitter_director_score
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",http://toystory.disney.com/toy-story,862,tt0114709,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",false,martial arts-jealousy-friendship-bullying-elem...,/lxD5ak7BOoinRNehOCA85CQ8ubr.jpg,...,7.966,16430.0,Tom Hanks-Tim Allen-Don Rickles-Jim Varney-Wal...,John Lasseter,False,NaN,"[Tom Hanks, ""Tim Allen, ""Don Rickles, ""Jim ...",0.000000,0.044497,0.312977
1,False,NaN,NaN,8844,tt0113497,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",false,giant insect-board game-jungle-disappearance-r...,/pYw10zrqfkdm3yD9JTO6vEGQhKy.jpg,...,7.237,9437.0,Robin Williams-Kirsten Dunst-Bradley Pierce-Bo...,Joe Johnston,False,NaN,"[Robin Williams, ""Kirsten Dunst, ""Bradley Pi...",0.000000,0.048193,0.236220
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,15602,tt0113228,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",false,fishing-halloween-sequel-old man-best friend-w...,/1J4Z7VhdAgtdd97nCxY7dcBpjGT.jpg,...,6.5,324.0,Walter Matthau-Jack Lemmon-Ann-Margret-Sophia ...,Howard Deutch,False,NaN,"[Walter Matthau, ""Jack Lemmon, ""Ann, ""Margr...",0.000000,0.481034,0.500000
3,False,NaN,NaN,31357,tt0114885,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",false,based on novel or book-interracial relationshi...,/jZjoEKXMTDoZAGdkjhAdJaKtXSN.jpg,...,6.3,128.0,Whitney Houston-Angela Bassett-Loretta Devine-...,Forest Whitaker,False,NaN,"[Whitney Houston, ""Angela Bassett, ""Loretta ...",0.000000,0.478686,-0.552932
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",NaN,11862,tt0113041,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",false,parent child relationship-baby-midlife crisis-...,/lEsjVrGU21BeJjF5AF9EWsihDpw.jpg,...,6.2,626.0,Steve Martin-Diane Keaton-Martin Short-Kimberl...,Charles Shyer,False,NaN,"[Steve Martin, ""Diane Keaton, ""Martin Short,...",0.000000,0.130087,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17737,false,None,,842945,tt10559102,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""english_name"": ""German"", ""iso_639_1"": ""de"",...",false,NaN,/m1fgGSLK0WvRpzM1AmZu38m0Tx8.jpg,...,6.332,1033.0,"[""Skeet Ulrich"", ""Anne Heche"", ""Daniel Diemer""...",Herbert James Winterstern,True,4.3,"[Skeet Ulrich, ""Anne Heche, ""Daniel Diemer, ...",0.433962,0.000000,0.000000
17738,false,None,,840326,tt14846026,"[{""iso_3166_1"": ""FI"", ""name"": ""Finland""}, {""is...","[{""english_name"": ""Finnish"", ""iso_639_1"": ""fi""...",false,NaN,/exI61quYa7xMfcIDSp674UnvrhG.jpg,...,7.106,4934.0,"[""Jorma Tommila"", ""Aksel Hennie"", ""Jack Doolan...",Jalmari Helander,True,7.3,"[Jorma Tommila, ""Aksel Hennie, ""Jack Doolan,...",0.512159,NaN,0.000000
17739,false,None,https://kotzmovie.com,455476,tt6528290,"[{""iso_3166_1"": ""JP"", ""name"": ""Japan""}, {""iso_...","[{""english_name"": ""English"", ""iso_639_1"": ""en""...",false,NaN,/qCW1DUmfWj2xLQRCRhrwkQyKROH.jpg,...,6.6,1660.0,"[""Famke Janssen"", ""Madison Iseman"", ""Sean Bean...",Tomasz Baginski,True,4.7,"[Famke Janssen, ""Madison Iseman, ""Sean Bean,...",0.392451,0.364508,0.000000
17740,false,None,https://www.renfieldmovie.com,649609,tt11358390,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""english_name"": ""English"", ""iso_639_1"": ""en""...",false,NaN,/e7FzphKs5gzoghDotAEp2FeP46u.jpg,...,7.251,19984.0,"[""Nicholas Hoult"", ""Nicolas Cage"", ""Awkwafina""...",Chris McKay,True,6.5,"[Nicholas Hoult, ""Nicolas Cage, ""Awkwafina, ...",NaN,NaN,NaN


In [53]:
if 'reddit_movie_score' not in df_final.columns:
    df_final = df_final.merge(df_reddit_movie_title[['id', 'title_l/d_ratio']], on='id', how='left', suffixes=('_movie', '_reddit'))
    df_final.rename(columns={'title_l/d_ratio': 'reddit_movie_score'}, inplace=True)   
    
if 'reddit_cast_average_score' not in df_final.columns:
    df_final = df_final.merge(df_reddit_average_score[['id', 'reddit_average_score']], on='id', how='left', suffixes=('_cast', '_reddit'))
    df_final.rename(columns={'reddit_average_score': 'reddit_cast_average_score'}, inplace=True)

if 'reddit_director_score' not in df_final.columns:
    df_final = df_final.merge(df_reddit_director[['id', 'director_l/d_ratio']], on='id', how='left', suffixes=('_director', '_reddit'))
    df_final.rename(columns={'director_l/d_ratio': 'reddit_director_score'}, inplace=True)   

# convert any lists in dataframe to tuples
df_final = df_final.applymap(lambda x: tuple(x) if isinstance(x, list) else x)

# drop duplicates
df_final.drop_duplicates(inplace=True)

# convert tuples back to lists
df_final = df_final.applymap(lambda x: list(x) if isinstance(x, tuple) else x)

df_final

,adult,belongs_to_collection,homepage,id,imdb_id,production_countries,spoken_languages,video,keywords,backdrop_path,...,director,New_status,rating,Top Cast,twitter_movie_score,twitter_cast_average_score,twitter_director_score,reddit_movie_score,reddit_cast_average_score,reddit_director_score
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",http://toystory.disney.com/toy-story,862,tt0114709,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",false,martial arts-jealousy-friendship-bullying-elem...,/lxD5ak7BOoinRNehOCA85CQ8ubr.jpg,...,John Lasseter,False,NaN,"[Tom Hanks, ""Tim Allen, ""Don Rickles, ""Jim ...",0.000000,0.044497,0.312977,0.436937,0.535185,0.281553
1,False,NaN,NaN,8844,tt0113497,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",false,giant insect-board game-jungle-disappearance-r...,/pYw10zrqfkdm3yD9JTO6vEGQhKy.jpg,...,Joe Johnston,False,NaN,"[Robin Williams, ""Kirsten Dunst, ""Bradley Pi...",0.000000,0.048193,0.236220,0.432225,0.440129,0.708738
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,15602,tt0113228,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",false,fishing-halloween-sequel-old man-best friend-w...,/1J4Z7VhdAgtdd97nCxY7dcBpjGT.jpg,...,Howard Deutch,False,NaN,"[Walter Matthau, ""Jack Lemmon, ""Ann, ""Margr...",0.000000,0.481034,0.500000,0.321429,0.532526,0.842105
3,False,NaN,NaN,31357,tt0114885,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",false,based on novel or book-interracial relationshi...,/jZjoEKXMTDoZAGdkjhAdJaKtXSN.jpg,...,Forest Whitaker,False,NaN,"[Whitney Houston, ""Angela Bassett, ""Loretta ...",0.000000,0.478686,-0.552932,0.566667,0.452171,0.596491
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",NaN,11862,tt0113041,"[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",false,parent child relationship-baby-midlife crisis-...,/lEsjVrGU21BeJjF5AF9EWsihDpw.jpg,...,Charles Shyer,False,NaN,"[Steve Martin, ""Diane Keaton, ""Martin Short,...",0.000000,0.130087,0.500000,0.462185,0.463424,0.866667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306312,false,None,,842945,tt10559102,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""english_name"": ""German"", ""iso_639_1"": ""de"",...",false,NaN,/m1fgGSLK0WvRpzM1AmZu38m0Tx8.jpg,...,Herbert James Winterstern,True,4.3,"[Skeet Ulrich, ""Anne Heche, ""Daniel Diemer, ...",0.433962,0.000000,0.000000,-0.333333,NaN,0.408696
306313,false,None,,840326,tt14846026,"[{""iso_3166_1"": ""FI"", ""name"": ""Finland""}, {""is...","[{""english_name"": ""Finnish"", ""iso_639_1"": ""fi""...",false,NaN,/exI61quYa7xMfcIDSp674UnvrhG.jpg,...,Jalmari Helander,True,7.3,"[Jorma Tommila, ""Aksel Hennie, ""Jack Doolan,...",0.512159,NaN,0.000000,0.561497,NaN,0.714286
306314,false,None,https://kotzmovie.com,455476,tt6528290,"[{""iso_3166_1"": ""JP"", ""name"": ""Japan""}, {""iso_...","[{""english_name"": ""English"", ""iso_639_1"": ""en""...",false,NaN,/qCW1DUmfWj2xLQRCRhrwkQyKROH.jpg,...,Tomasz Baginski,True,4.7,"[Famke Janssen, ""Madison Iseman, ""Sean Bean,...",0.392451,0.364508,0.000000,0.303371,NaN,0.520000
306315,false,None,https://www.renfieldmovie.com,649609,tt11358390,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""english_name"": ""English"", ""iso_639_1"": ""en""...",false,NaN,/e7FzphKs5gzoghDotAEp2FeP46u.jpg,...,Chris McKay,True,6.5,"[Nicholas Hoult, ""Nicolas Cage, ""Awkwafina, ...",NaN,NaN,NaN,NaN,NaN,NaN


#### Write data to the database

In [54]:
# Function to convert values to JSON based on their types
def convert_to_json(value):
    if isinstance(value, (np.integer, int, float, bool)):
        return json.dumps(value)
    elif isinstance(value, (list, dict)):
        return json.dumps(value)
    else:
        return value

# Save the DataFrame to the PostgreSQL database
table_name = "movie_merged_data"

# Create a new DataFrame with converted values
converted_df = df_final.applymap(convert_to_json)

# Save the converted DataFrame to the PostgreSQL database
converted_df.to_sql(
    table_name,
    engine,
    if_exists="replace",  # specify how to handle if the table already exists
    index=False  
)

626

In [55]:
# Execute a SELECT query to retrieve data from the table
cur.execute("SELECT * FROM movie_merged_data")

# Fetch the top 10 rows from the result set
rows = cur.fetchmany(100)

# Get the column names from the description of the cursor
columns = [desc[0] for desc in cur.description]

# Output the column names
print(columns)

# Output the contents of the table (top 10 rows)
for row in rows:
    print(row)

['adult', 'belongs_to_collection', 'homepage', 'id', 'imdb_id', 'production_countries', 'spoken_languages', 'video', 'keywords', 'backdrop_path', 'recommendations', 'budget', 'genres', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'release_date', 'revenue', 'runtime', 'status', 'tagline', 'vote_average', 'vote_count', 'cast', 'director', 'New_status', 'rating', 'Top Cast', 'twitter_movie_score', 'twitter_cast_average_score', 'twitter_director_score', 'reddit_movie_score', 'reddit_cast_average_score', 'reddit_director_score']
('False', "{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}", 'http://toystory.disney.com/toy-story', '862', 'tt0114709', "[{'iso_3166_1': 'US', 'name': 'United States of America'}]", "[{'iso_639_1': 'en', 'name': 'English'}]", 'false', 'martial arts-jealousy-friendship-bullying-elementary school-friends-riva